In [33]:
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast

#import torch.nn as nn

from scipy.special import softmax
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import urllib
import csv
import json
import glob
import os


from tensorflow.python.client import device_lib
import multiprocessing
print(multiprocessing.cpu_count())
device_lib.list_local_devices()

12


2023-05-30 11:27:45.222222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 11:27:45.223565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 11:27:45.224137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 11:27:45.224890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 11:27:45.225472: I tensorflow/compiler/xla/stream_executo

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16417088075855329816
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2603876352
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15629598123103945116
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [3]:
!wget https://dem-primary-tweets.s3.amazonaws.com/PeteForAmerica.1574004110.txt


--2023-05-30 10:12:11--  https://dem-primary-tweets.s3.amazonaws.com/PeteForAmerica.1574004110.txt
Resolving dem-primary-tweets.s3.amazonaws.com (dem-primary-tweets.s3.amazonaws.com)... 52.216.60.249, 52.217.100.60, 52.217.204.49, ...
Connecting to dem-primary-tweets.s3.amazonaws.com (dem-primary-tweets.s3.amazonaws.com)|52.216.60.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15936465 (15M) [text/plain]
Saving to: ‘PeteForAmerica.1574004110.txt’

PeteForAmerica.1574 100%[===================>]  15.20M  7.64MB/s    in 2.0s    

2023-05-30 10:12:14 (7.64 MB/s) - ‘PeteForAmerica.1574004110.txt’ saved [15936465/15936465]



In [5]:
df = pd.read_json('PeteForAmerica.1574004110.txt', lines=True)
df

,contributors,coordinates,created_at,display_text_range,entities,favorite_count,favorited,full_text,geo,id,...,retweeted_status,source,truncated,user,possibly_sensitive,extended_entities,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink
0,NaN,NaN,2019-11-17 15:15:34+00:00,"[0, 140]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",0,False,RT @PeteButtigieg: I am not just here to end t...,NaN,1196084469637234688,...,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/iphone"" r...",False,"{'can_media_tag': True, 'contributors_enabled'...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2019-11-17 15:00:00+00:00,"[0, 98]","{'hashtags': [], 'symbols': [], 'urls': [{'dis...",211,False,“The president has again dishonored our armed ...,NaN,1196080551666409473,...,NaN,"<a href=""https://about.twitter.com/products/tw...",False,"{'can_media_tag': True, 'contributors_enabled'...",0.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2019-11-17 02:00:00+00:00,"[0, 14]","{'hashtags': [], 'media': [{'display_url': 'pi...",5168,False,Win with hope. https://t.co/JI1ifDCg5Z,NaN,1195884258017075205,...,NaN,"<a href=""https://about.twitter.com/products/tw...",False,"{'can_media_tag': True, 'contributors_enabled'...",0.0,{'media': [{'display_url': 'pic.twitter.com/JI...,NaN,NaN,NaN,NaN
3,NaN,NaN,2019-11-17 00:45:55+00:00,"[0, 140]","{'hashtags': [{'indices': [77, 88], 'text': 'R...",0,False,RT @UniNoticias: El turno de @PeteButtigieg: a...,NaN,1195865616533639170,...,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/iphone"" r...",False,"{'can_media_tag': True, 'contributors_enabled'...",0.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2019-11-17 00:37:15+00:00,"[0, 138]","{'hashtags': [], 'symbols': [], 'urls': [{'dis...",258,False,Don’t miss @PeteButtigieg speak about his visi...,NaN,1195863434119843842,...,NaN,"<a href=""https://about.twitter.com/products/tw...",False,"{'can_media_tag': True, 'contributors_enabled'...",0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332,NaN,NaN,2019-09-03 15:25:27+00:00,"[0, 140]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",0,False,RT @Chas10Buttigieg: As we begin to open 20 ne...,NaN,1168907865534468096,...,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/iphone"" r...",False,"{'can_media_tag': True, 'contributors_enabled'...",NaN,NaN,NaN,1.168851e+18,1.168851e+18,"{'display': 'twitter.com/PeteForAmerica…', 'ex..."
2333,NaN,NaN,2019-09-03 15:15:00+00:00,"[0, 182]","{'hashtags': [{'indices': [4, 13], 'text': 'Pe...",313,False,The #PeteWave is coming to New Hampshire! Join...,NaN,1168905236737646597,...,NaN,"<a href=""https://about.twitter.com/products/tw...",False,"{'can_media_tag': True, 'contributors_enabled'...",0.0,NaN,NaN,NaN,NaN,NaN
2334,NaN,NaN,2019-09-03 14:18:41+00:00,"[0, 74]","{'hashtags': [{'indices': [10, 21], 'text': 'P...",1150,False,Hyped for #PhaseThree?\n\nBecome a volunteer t...,NaN,1168891064314408961,...,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'can_media_tag': True, 'contributors_enabled'...",0.0,{'media': [{'additional_media_info': {'monetiz...,NaN,NaN,NaN,NaN
2335,NaN,NaN,2019-09-03 12:46:48+00:00,"[0, 140]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",0,False,"RT @PeteButtigieg: 20 new offices in Iowa, 12 ...",NaN,1168867942299312131,...,"{'contributors': None, 'coordinates': None, 'c...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,"{'can_media_tag': True, 'contributors_enabled'...",NaN,NaN,NaN,1.168851e+18,1.168851e+18,"{'display': 'twitter.com/PeteForAmerica…', 'ex..."


In [10]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
 
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pt_batch = tokenizer(
["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt")
 
pt_outputs = pt_model(**pt_batch)
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=1)
pt_predictions

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward0>)

In [30]:
#device_ids = [0,1,2,3]
dev = 'cuda'
if dev == 'cpu':
  device = torch.device('cpu')
  device_staging = 'cpu:0'
else:
  device = torch.device('cuda')
  device_staging = 'cuda:0'

In [31]:
# Get the number of available GPUs
num_gpus = torch.cuda.device_count()

# Create a list of available GPU device IDs
device_ids = []
for i in range(num_gpus):
    device_ids.append(i)

print(device_ids)

[0]


In [32]:
# Get the device name
device_name = torch.cuda.get_device_name('cuda:0')

print(device_name)

NVIDIA GeForce GTX 1650


In [35]:
import torch
import torch.nn as nn
from torch.cuda.amp import autocast
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import pandas as pd

MODEL = "distilbert-base-uncased-finetuned-sst-2-english"

def get_all_files():
    file_list = ['PeteForAmerica.1574004110.txt']
    return file_list


class TextLoader(Dataset):
    def __init__(self, file=None, transform=None, target_transform=None, tokenizer=None):
        self.file = pd.read_json(file, lines=True)
        self.file = self.file
        self.file = tokenizer(list(self.file['full_text']), padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.file = self.file['input_ids']
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.file)

    def __getitem__(self, idx):
        data = self.file[idx]
        return data


class SentimentModel(nn.Module):
    def __init__(self):
        super(SentimentModel, self).__init__()
        self.fc = AutoModelForSequenceClassification.from_pretrained(MODEL)

    def forward(self, input):
        output = self.fc(input)
        pt_predictions = nn.functional.softmax(output.logits, dim=1)
        return pt_predictions


dev = 'cuda'
if dev == 'cpu':
    device = torch.device('cpu')
    device_staging = 'cpu'
else:
    device = torch.device('cuda')
    device_staging = 'cuda:0'

tokenizer = AutoTokenizer.from_pretrained(MODEL)

all_files = get_all_files()
model3 = SentimentModel().to(device_staging)
model3 = nn.DataParallel(model3)  # Use all available GPUs

try:
    scaler = torch.cuda.amp.GradScaler()  # Create a GradScaler for mixed-precision training
    torch.set_printoptions(threshold=10000)

    for file in all_files:
        data = TextLoader(file=file, tokenizer=tokenizer)
        train_dataloader = DataLoader(data, batch_size=32, shuffle=False)
        out = torch.empty(0, 0).to(device_staging)

        for data in train_dataloader:
            input = data.to(device_staging)

            # Enable mixed-precision training
            with torch.cuda.amp.autocast():
                out = model3(input)

            # Rest of your code...

        df = pd.read_json(file, lines=True)['full_text']
        res = out.cpu().numpy()
        df_res = pd.DataFrame({"text": df, "negative": res[:, 0], "positive": res[:, 1]})
        print(df_res)

except Exception as e:
    print(f"An error occurred: {str(e)}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 3.82 GiB total capacity; 2.71 GiB already allocated; 33.25 MiB free; 2.82 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF